In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

support_data_path = "./Datasets/Support Datasets/"
ars_data_path = "./Datasets/Statistikportal DE/"
zensus_data_path = "./Datasets/Zensus/"
zensus_csv_data_path = "./Datasets/Zensus/csv_sheets_export/"

df = pd.read_excel(f"{ars_data_path}Anschriften_Regionalschluessel.xlsx", engine="openpyxl")

### Regionalschlüssel auslesen ✅

Es werden nur noch benötigte Spalten behalten und die Spaltenköpfe werden bennant.

In [2]:
df_RS = df.iloc[:, [1, 4, 5, 6, 7, 10, 11]]
df_RS.columns = ['Land', 'Textkennzeichen', 'ARS', 'Amtlicher Gemeindeschlüssel(AGS)', 'Gemeinde/Stadt', 'PLZ', 'Ort']

In [3]:
x = 7  # Die ersten 7 Zeilen entfernen (keine echten Daten enthalten)
df_RS = df_RS.iloc[x:]
df_RS.head()

,Land,Textkennzeichen,ARS,Amtlicher Gemeindeschlüssel(AGS),Gemeinde/Stadt,PLZ,Ort
7,Schleswig-Holstein,NaN,01,NaN,Schleswig-Holstein,24105,Kiel
8,Schleswig-Holstein,Kreisfreie Stadt,01001,NaN,"Flensburg, Stadt",24931,Flensburg
9,Schleswig-Holstein,Kreisfreie Stadt,010010000000,01001000,"Flensburg, Stadt",24937,Flensburg
10,Schleswig-Holstein,Kreisfreie Stadt,01002,NaN,"Kiel, Landeshauptstadt",24103,Kiel
11,Schleswig-Holstein,Kreisfreie Stadt,010020000000,01002000,"Kiel, Landeshauptstadt",24103,Kiel


Das Ergebnis ist eine Tabelle mit dem Regionalschlüssel, sowie PLZ und Ort.

In [4]:
# Export als CSV
df_RS.to_csv(f"{support_data_path}plz_ars.csv",sep=',',index=False,header=True,encoding='utf-8')

### Schleife Join ✅

In [5]:
import glob
import pandas as pd
import os

df_RS = pd.read_csv(support_data_path + "plz_ars.csv", dtype={'ARS': str, 'PLZ': str})
df_RS['ARS'] = df_RS['ARS'].astype(str).str.zfill(12)

# Alle CSV-Dateien im Ordner durchgehen (liefert eine Liste von Strings)
for csvfile in glob.glob(zensus_csv_data_path + '*.csv'):
    print(f"Bearbeite: {os.path.basename(csvfile)}")

    df = pd.read_csv(csvfile, dtype={'_RS': str})
    df['_RS'] = df['_RS'].astype(str).str.zfill(12)
    df_merged = df.merge(df_RS[['ARS', 'PLZ', 'Ort']], left_on='_RS', right_on='ARS', how='left')
    df_merged = df_merged.drop(columns=['ARS'])

    # Suffix "_mitPLZ.csv" anhängen
    outname = csvfile.replace('.csv', '_mitPLZ.csv')
    df_merged.to_csv(outname, index=False, encoding='utf-8')

    print(f"→ Fertig: {os.path.basename(outname)}")


Bearbeite: Regionaltabelle_Bevoelkerung_CSV.csv
→ Fertig: Regionaltabelle_Bevoelkerung_CSV_mitPLZ.csv
Bearbeite: Regionaltabelle_Bevoelkerung_CSV_mitPLZ.csv
→ Fertig: Regionaltabelle_Bevoelkerung_CSV_mitPLZ_mitPLZ.csv
Bearbeite: Regionaltabelle_Bildung_Erwerbstaetigkeit_CSV.csv
→ Fertig: Regionaltabelle_Bildung_Erwerbstaetigkeit_CSV_mitPLZ.csv
Bearbeite: Regionaltabelle_Bildung_Erwerbstaetigkeit_CSV_mitPLZ.csv
→ Fertig: Regionaltabelle_Bildung_Erwerbstaetigkeit_CSV_mitPLZ_mitPLZ.csv
Bearbeite: Regionaltabelle_Demografie_CSV.csv
→ Fertig: Regionaltabelle_Demografie_CSV_mitPLZ.csv
Bearbeite: Regionaltabelle_Demografie_CSV_mitPLZ.csv
→ Fertig: Regionaltabelle_Demografie_CSV_mitPLZ_mitPLZ.csv
Bearbeite: Regionaltabelle_Gebaeude_Wohnungen_CSV.csv
→ Fertig: Regionaltabelle_Gebaeude_Wohnungen_CSV_mitPLZ.csv
Bearbeite: Regionaltabelle_Gebaeude_Wohnungen_CSV_mitPLZ.csv
→ Fertig: Regionaltabelle_Gebaeude_Wohnungen_CSV_mitPLZ_mitPLZ.csv
Bearbeite: Regionaltabelle_Haushalte_CSV.csv
→ Fertig: Regio

### Sprechende Bezeichnungen für Zensus ✅

#### Säubern der Mapping Dateien ✅

Die Mapping-Tabellen bspw. `Regionaltabelle_Bevoelkerung_Mapping.csv` wurden aus den jeweiligen xlsx gezogen. Dort gibt es Sheets namens "Erläuterung zu CSV-Tabellen" diese wurden händisch extrahiert und in die Regionaltabelle_XXX_Mapping.csv geschrieben. Diese werden zunächst gesäubert, da dort noch irrelevante Sonderzeichen enthalten sind.

In [6]:
import os
import re

# Mapping-Ordner
map_ordner = "./Datasets/Zensus/Mapping/"

# Finde alle Mapping-Tabellen
mapping_files = [f for f in os.listdir(map_ordner) if f.endswith('_Mapping.csv')]

for fname in mapping_files:
    full_in = os.path.join(map_ordner, fname)
    full_out = os.path.join(map_ordner, fname.replace('.csv', '_clean.csv'))
    
    with open(full_in, encoding='utf-8') as fin, open(full_out, 'w', encoding='utf-8') as fout:
        for line in fin:
            # Entfernt „– “ am Zeilenanfang (Unicode-Gedankenstrich + Leerzeichen)
            cleaned = re.sub(r'^[–-]\s+', '', line)
            fout.write(cleaned)
    print(f"Bereinigt: {full_out}")

print("Alle Mapping-Tabellen wurden gesäubert.")


Bereinigt: ./Datasets/Zensus/Mapping/Regionaltabelle_Bevoelkerung_Mapping_clean.csv
Bereinigt: ./Datasets/Zensus/Mapping/Regionaltabelle_Bildung_Erwerbstaetigkeit_Mapping_clean.csv
Bereinigt: ./Datasets/Zensus/Mapping/Regionaltabelle_Demografie_Mapping_clean.csv
Bereinigt: ./Datasets/Zensus/Mapping/Regionaltabelle_Gebaeude_Wohnungen_Mapping_clean.csv
Bereinigt: ./Datasets/Zensus/Mapping/Regionaltabelle_Haushalte_Mapping_clean.csv
Alle Mapping-Tabellen wurden gesäubert.


Hier findet das eigentliche Mapping statt

In [7]:
import os
import pandas as pd

# Ordner mit den Regionaltabellen
datapath_regionaltabellen = zensus_csv_data_path

# Mapping-Ordner (bitte den Pfad passend zu deiner Struktur)
mapping_folder = map_ordner

output_path_csv = "./Datasets/Zensus/Regionaltabellen/"

# Hole alle relevanten CSV-Dateien
regional_csvs = [f for f in os.listdir(datapath_regionaltabellen) if f.endswith("_CSV_mitPLZ.csv")]

for csv_file in regional_csvs:
    base_name = csv_file.replace("_CSV_mitPLZ.csv", "")
    mapping_file = f"{base_name}_Mapping_clean.csv"
    csv_path = os.path.join(datapath_regionaltabellen, csv_file)
    mapping_path = os.path.join(mapping_folder, mapping_file)

    if not os.path.exists(mapping_path):
        print(f"Mapping nicht gefunden für: {csv_file}")
        continue

    df = pd.read_csv(csv_path, dtype=str)

    # Mapping laden: Zwei Spalten, erste Zeile überspringen, eigene Spaltennamen
    mapping = pd.read_csv(mapping_path, dtype=str, header=None, skiprows=1, names=["alt", "neu"])

    # Optionale Filter: nur Zeilen mit echten Werten und keine Kommentare
    mapping = mapping[mapping["alt"].notnull() & mapping["neu"].notnull()]
    mapping = mapping[~mapping["alt"].str.startswith("–")]

    # Mapping-Dictionary bauen
    col_map = dict(zip(mapping["alt"], mapping["neu"]))

    # Spalten umbenennen (Berichtszeitpunkt bleibt, alles andere wird gemappt)
    new_columns = [col if col == "Berichtszeitpunkt" else col_map.get(col, col) for col in df.columns]
    df.columns = new_columns

    # Datei speichern
    output_path = os.path.join(output_path_csv, f"{base_name}_CSV_mitPLZ_final.csv")
    df.to_csv(output_path, index=False, encoding="utf-8-sig")
    print(f"Datei geschrieben: {output_path}")

print("Alle Tabellen wurden verarbeitet.")


Datei geschrieben: ./Datasets/Zensus/Regionaltabellen/Regionaltabelle_Bevoelkerung_CSV_mitPLZ_final.csv
Datei geschrieben: ./Datasets/Zensus/Regionaltabellen/Regionaltabelle_Bildung_Erwerbstaetigkeit_CSV_mitPLZ_final.csv
Datei geschrieben: ./Datasets/Zensus/Regionaltabellen/Regionaltabelle_Demografie_CSV_mitPLZ_final.csv
Datei geschrieben: ./Datasets/Zensus/Regionaltabellen/Regionaltabelle_Gebaeude_Wohnungen_CSV_mitPLZ_final.csv
Datei geschrieben: ./Datasets/Zensus/Regionaltabellen/Regionaltabelle_Haushalte_CSV_mitPLZ_final.csv
Alle Tabellen wurden verarbeitet.
